In [1]:
import requests,numpy,pandas,json,matplotlib,datetime,time
from flask import Flask
from io import StringIO

ImportError: No module named requests

### Data requesting function

In [17]:
def create_new_header(orignal_headers):
    new_headers = []

    for column in orignal_headers:
        data = str(column)
        if data == "證券代號":
            new_headers.append("stock_symbol")
        elif data == "日期":
            new_headers.append("date")
        elif data == "證券名稱":
            new_headers.append("stock_name")
        elif data == "成交股數":
            new_headers.append("volume")
        elif data == "成交金額":
            new_headers.append("total_price")
        elif data == "開盤價":
            new_headers.append("open")
        elif data == "最高價":
            new_headers.append("high")
        elif data == "最低價":
            new_headers.append("low")
        elif data == "收盤價":
            new_headers.append("close")
        elif data == "漲跌價差":
            new_headers.append("spread")
        elif data == "成交筆數":
            new_headers.append("transactions_number")

    return new_headers

def convert_date(data_ROC):
    """因為格式為109/1/1，為民國年，需轉換成西元年"""
    date_arr = data_ROC.split("/")
    new_year = int(date_arr[0]) + 1911
    return f"{new_year}-{date_arr[1]}-{date_arr[2]}"

def get_stock_history_data(stock_symbol, his_month):
    # API位置
    # address = "http://www.twse.com.tw/exchangeReport/STOCK_DAY_ALL?response=open_data"
    
    stock = stock_symbol
    date = his_month
    address = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date}&stockNo={stock}"

    # 取得資料
    response = requests.get(address)
    # 解析
    # 有幾個部分：stat, date, title, fields, data, notes
    data = response.text  # 這是json格式的資料
    a_json = json.loads(data)  # 轉成dict
    df = pandas.DataFrame.from_dict(a_json["data"])  # 轉成dataframe

    # 修改欄位名稱
    new_headers = create_new_header(a_json["fields"])
    df.columns = new_headers  # 設定資料欄位的名稱
    # print(df)

    df.to_csv(f"{stock_symbol}_{his_month}.csv")
    print("===finished===")


### Data analysis funciton

### Requesting data

date_list = (
    pandas.date_range(
        "2014-2-1",
        datetime.datetime.now().strftime("%Y-%m-%d"),
        freq="MS",
        tz="Asia/Taipei",
    )
    .strftime("%Y%m%d")
    .tolist()
)
stock_list=["2330","0050"]
print("date list:")
print(date_list)
print("stock list:")
print(stock_list)
for data_stock in stock_list:
    for data_date in date_list:
        print(f"start({data_stock}): {data_date}")
        get_stock_history_data(data_stock, data_date)
        time.sleep(1)
    #bug: https://stackoverflow.com/questions/45529909/python-request-futures-does-not-seem-to-be-working-for-me
    #need to limit the request, solution vpngate.py


In [ ]:
# https://ithelp.ithome.com.tw/articles/10248648
# https://www.finlab.tw/Python-%E7%B0%A1%E5%96%AE158%E7%A8%AE%E6%8A%80%E8%A1%93%E6%8C%87%E6%A8%99%E8%A8%88%E7%AE%97/#KD_zhi_ji_suan